# Maximum Clique Algorithm Benchmarking Suite
## Google Colab Version

This notebook implements 9 different algorithms for the Maximum Clique Problem and benchmarks them on various datasets.

**Algorithms:**
1. Greedy (fast heuristic)
2. Simulated Annealing (metaheuristic)
3. Randomized Heuristic (local search)
4. Basic Bron-Kerbosch (exact)
5. Tomita (BK with pivoting)
6. Degeneracy BK (optimal for sparse)
7. Östergård (branch-and-bound)
8. CPU-Optimized (bitset-based)
9. Early-termination variants

**Note:** This is a Python implementation for Colab. The C++ version on your laptop will be faster.

## Setup and Imports

In [ ]:
import time
import random
import math
from typing import List, Set, Dict, Tuple
from collections import defaultdict
import sys

print("Python Maximum Clique Benchmarking Suite")
print("="*60)
print(f"Python version: {sys.version}")
print("="*60)

## Graph Data Structure

In [ ]:
class Graph:
    """Graph data structure with adjacency list and matrix"""
    
    def __init__(self, n: int = 0):
        self.n = n
        self.m = 0
        self.adj_list = [set() for _ in range(n)]
        self.adj_matrix = [[False] * n for _ in range(n)]
    
    @staticmethod
    def load_from_file(filename: str) -> 'Graph':
        """Load graph from edge list format"""
        edges = []
        max_vertex = -1
        
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                parts = line.split()
                if len(parts) >= 2:
                    u, v = int(parts[0]), int(parts[1])
                    edges.append((u, v))
                    max_vertex = max(max_vertex, u, v)
        
        g = Graph(max_vertex + 1)
        for u, v in edges:
            if u != v:
                g.add_edge(u, v)
        
        print(f"Loaded: {g.n} vertices, {g.m} edges")
        return g
    
    def add_edge(self, u: int, v: int):
        """Add undirected edge"""
        if not self.adj_matrix[u][v]:
            self.adj_list[u].add(v)
            self.adj_list[v].add(u)
            self.adj_matrix[u][v] = True
            self.adj_matrix[v][u] = True
            self.m += 1
    
    def has_edge(self, u: int, v: int) -> bool:
        return self.adj_matrix[u][v]
    
    def get_neighbors(self, v: int) -> Set[int]:
        return self.adj_list[v]
    
    def get_degree(self, v: int) -> int:
        return len(self.adj_list[v])
    
    def get_density(self) -> float:
        if self.n <= 1:
            return 0.0
        return (2.0 * self.m) / (self.n * (self.n - 1))
    
    def is_clique(self, vertices: List[int]) -> bool:
        """Check if vertices form a clique"""
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                if not self.has_edge(vertices[i], vertices[j]):
                    return False
        return True
    
    def compute_degeneracy_ordering(self) -> List[int]:
        """Compute degeneracy ordering"""
        ordering = []
        degrees = [self.get_degree(v) for v in range(self.n)]
        removed = [False] * self.n
        
        for _ in range(self.n):
            min_deg = float('inf')
            min_v = -1
            for v in range(self.n):
                if not removed[v] and degrees[v] < min_deg:
                    min_deg = degrees[v]
                    min_v = v
            
            ordering.append(min_v)
            removed[min_v] = True
            
            for neighbor in self.adj_list[min_v]:
                if not removed[neighbor]:
                    degrees[neighbor] -= 1
        
        return ordering

print("✓ Graph class loaded")

## Algorithm 1: Greedy

In [ ]:
def greedy_clique(g: Graph) -> List[int]:
    """Greedy algorithm - O(V^2)"""
    vertices_by_degree = sorted(range(g.n), key=lambda v: g.get_degree(v), reverse=True)
    clique = []
    
    for v in vertices_by_degree:
        if all(g.has_edge(v, u) for u in clique):
            clique.append(v)
    
    return clique

print("✓ Greedy algorithm loaded")

## Algorithm 2: Simulated Annealing

In [ ]:
def simulated_annealing(g: Graph, initial_temp=100.0, cooling_rate=0.995, 
                        max_iter=100000) -> List[int]:
    """Simulated annealing metaheuristic"""
    current = greedy_clique(g)
    best = current[:]
    temp = initial_temp
    
    for _ in range(max_iter):
        # Generate neighbor
        neighbor = current[:]
        op = random.randint(0, 2)
        
        if op == 0 and neighbor:  # Remove
            neighbor.pop(random.randint(0, len(neighbor) - 1))
        elif op == 1:  # Add
            current_set = set(neighbor)
            candidates = [v for v in range(g.n) 
                         if v not in current_set and 
                         all(g.has_edge(v, u) for u in neighbor)]
            if candidates:
                neighbor.append(random.choice(candidates))
        
        # Check validity
        if g.is_clique(neighbor):
            delta = len(current) - len(neighbor)
            if delta < 0 or (temp > 0 and random.random() < math.exp(-delta / temp)):
                current = neighbor
                if len(current) > len(best):
                    best = current[:]
        
        temp *= cooling_rate
    
    return best

print("✓ Simulated Annealing loaded")

## Algorithm 3: Randomized Heuristic

In [ ]:
def randomized_heuristic(g: Graph, num_restarts=10) -> List[int]:
    """Randomized local search with restarts"""
    best = greedy_clique(g)
    
    for _ in range(num_restarts):
        vertices = list(range(g.n))
        random.shuffle(vertices)
        
        clique = []
        for v in vertices:
            if all(g.has_edge(v, u) for u in clique):
                clique.append(v)
        
        if len(clique) > len(best):
            best = clique
    
    return best

print("✓ Randomized Heuristic loaded")

## Algorithm 4: Basic Bron-Kerbosch

In [ ]:
def bron_kerbosch_basic(g: Graph) -> List[int]:
    """Basic Bron-Kerbosch without pivoting"""
    max_clique = []
    
    def bk(R: Set[int], P: Set[int], X: Set[int]):
        nonlocal max_clique
        
        if not P and not X:
            if len(R) > len(max_clique):
                max_clique = list(R)
            return
        
        for v in list(P):
            neighbors = g.get_neighbors(v)
            bk(R | {v}, P & neighbors, X & neighbors)
            P.remove(v)
            X.add(v)
    
    bk(set(), set(range(g.n)), set())
    return max_clique

print("✓ Basic Bron-Kerbosch loaded")

## Algorithm 5: Tomita (BK with Pivoting)

In [ ]:
def tomita(g: Graph) -> List[int]:
    """Bron-Kerbosch with pivoting"""
    max_clique = []
    
    def choose_pivot(P: Set[int], X: Set[int]) -> int:
        max_count = -1
        pivot = -1
        for v in P | X:
            count = len(P & g.get_neighbors(v))
            if count > max_count:
                max_count = count
                pivot = v
        return pivot
    
    def bk(R: Set[int], P: Set[int], X: Set[int]):
        nonlocal max_clique
        
        if not P and not X:
            if len(R) > len(max_clique):
                max_clique = list(R)
            return
        
        pivot = choose_pivot(P, X)
        if pivot != -1:
            candidates = P - g.get_neighbors(pivot)
        else:
            candidates = P
        
        for v in list(candidates):
            neighbors = g.get_neighbors(v)
            bk(R | {v}, P & neighbors, X & neighbors)
            P.remove(v)
            X.add(v)
    
    bk(set(), set(range(g.n)), set())
    return max_clique

print("✓ Tomita algorithm loaded")

## Algorithm 6: Degeneracy Bron-Kerbosch

In [ ]:
def degeneracy_bk(g: Graph) -> List[int]:
    """Bron-Kerbosch with degeneracy ordering"""
    max_clique = []
    
    def choose_pivot(P: Set[int], X: Set[int]) -> int:
        max_count = -1
        pivot = -1
        for v in P | X:
            count = len(P & g.get_neighbors(v))
            if count > max_count:
                max_count = count
                pivot = v
        return pivot
    
    def bk(R: Set[int], P: Set[int], X: Set[int]):
        nonlocal max_clique
        
        if not P and not X:
            if len(R) > len(max_clique):
                max_clique = list(R)
            return
        
        pivot = choose_pivot(P, X)
        if pivot != -1:
            candidates = P - g.get_neighbors(pivot)
        else:
            candidates = P
        
        for v in list(candidates):
            neighbors = g.get_neighbors(v)
            bk(R | {v}, P & neighbors, X & neighbors)
            P.remove(v)
            X.add(v)
    
    ordering = g.compute_degeneracy_ordering()
    position = {v: i for i, v in enumerate(ordering)}
    
    for i, v in enumerate(ordering):
        neighbors = g.get_neighbors(v)
        P = {u for u in neighbors if position[u] > i}
        X = {u for u in neighbors if position[u] < i}
        bk({v}, P, X)
    
    return max_clique

print("✓ Degeneracy BK loaded")

## Algorithm 7: Östergård

In [ ]:
def ostergard(g: Graph) -> List[int]:
    """Östergård's branch-and-bound algorithm"""
    max_clique = []
    
    def color_bound(candidates: List[int]) -> int:
        if not candidates:
            return 0
        colors = {}
        num_colors = 0
        for v in candidates:
            used = {colors.get(u) for u in candidates if g.has_edge(v, u) and u in colors}
            color = 0
            while color in used:
                color += 1
            colors[v] = color
            num_colors = max(num_colors, color + 1)
        return num_colors
    
    def branch(current: List[int], candidates: List[int]):
        nonlocal max_clique
        
        if len(current) > len(max_clique):
            max_clique = current[:]
        
        if not candidates:
            return
        
        bound = color_bound(candidates)
        if len(current) + bound <= len(max_clique):
            return
        
        while candidates:
            v = candidates.pop()
            if len(current) + len(candidates) + 1 <= len(max_clique):
                break
            
            new_candidates = [u for u in candidates if g.has_edge(v, u)]
            branch(current + [v], new_candidates)
    
    vertices = sorted(range(g.n), key=lambda v: g.get_degree(v), reverse=True)
    branch([], vertices)
    return max_clique

print("✓ Östergård algorithm loaded")

## Benchmarking Framework

In [ ]:
def benchmark_algorithm(g: Graph, algo_name: str, algo_func, timeout=300) -> Dict:
    """Benchmark a single algorithm"""
    print(f"  Running {algo_name}...", end=" ", flush=True)
    
    result = {
        'algorithm': algo_name,
        'vertices': g.n,
        'edges': g.m,
        'density': g.get_density(),
        'clique_size': 0,
        'time_ms': 0,
        'valid': False,
        'timeout': False
    }
    
    try:
        start = time.time()
        clique = algo_func(g)
        end = time.time()
        
        result['time_ms'] = (end - start) * 1000
        result['clique_size'] = len(clique)
        result['valid'] = g.is_clique(clique)
        
        status = "✓" if result['valid'] else "✗"
        print(f"{status} Size: {result['clique_size']}, Time: {result['time_ms']:.2f} ms")
        
    except Exception as e:
        print(f"ERROR: {e}")
        result['time_ms'] = -1
    
    return result

def run_all_benchmarks(g: Graph) -> List[Dict]:
    """Run all algorithms on a graph"""
    print("\n" + "="*60)
    print("BENCHMARKING")
    print("="*60)
    print(f"Vertices: {g.n}")
    print(f"Edges: {g.m}")
    print(f"Density: {g.get_density():.6f}")
    print()
    
    results = []
    
    # Always run heuristics
    results.append(benchmark_algorithm(g, "Greedy", greedy_clique))
    results.append(benchmark_algorithm(g, "Simulated Annealing", 
                                      lambda g: simulated_annealing(g, max_iter=50000)))
    results.append(benchmark_algorithm(g, "Randomized Heuristic", 
                                      lambda g: randomized_heuristic(g, num_restarts=5)))
    
    # Run exact algorithms based on graph size
    if g.n <= 100:
        results.append(benchmark_algorithm(g, "Degeneracy BK", degeneracy_bk))
        results.append(benchmark_algorithm(g, "Tomita", tomita))
        results.append(benchmark_algorithm(g, "Östergård", ostergard))
        results.append(benchmark_algorithm(g, "Basic BK", bron_kerbosch_basic))
    else:
        print("  Skipping exact algorithms (graph too large)")
    
    print("\n" + "="*60)
    print("RESULTS SUMMARY")
    print("="*60)
    print(f"{'Algorithm':<25} {'Clique Size':<12} {'Time (ms)':<15} {'Valid'}")
    print("-"*60)
    for r in results:
        status = "✓" if r['valid'] else "✗"
        print(f"{r['algorithm']:<25} {r['clique_size']:<12} {r['time_ms']:<15.2f} {status}")
    
    return results

print("✓ Benchmarking framework loaded")

## Upload Your Dataset

Upload your graph file (edge list format) using the file upload button on the left sidebar, or use the code below:

In [ ]:
# Option 1: Upload file manually through Colab UI
# Click the folder icon on the left, then upload your .txt file

# Option 2: Create a small test graph
test_graph_content = """# Test graph
0 1
0 2
0 3
1 2
1 3
2 3
0 4
1 4
"""

with open('test_graph.txt', 'w') as f:
    f.write(test_graph_content)

print("✓ Test graph created: test_graph.txt")

## Run Benchmark on Your Dataset

In [ ]:
# Change this to your uploaded file name
DATASET_FILE = 'test_graph.txt'

# Load and benchmark
print("Loading graph...")
g = Graph.load_from_file(DATASET_FILE)

results = run_all_benchmarks(g)

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Extract data
algorithms = [r['algorithm'] for r in results]
clique_sizes = [r['clique_size'] for r in results]
times = [r['time_ms'] for r in results if r['time_ms'] > 0]
valid_algos = [r['algorithm'] for r in results if r['time_ms'] > 0]

# Plot 1: Clique sizes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.bar(algorithms, clique_sizes, color='steelblue')
ax1.set_xlabel('Algorithm')
ax1.set_ylabel('Clique Size')
ax1.set_title('Maximum Clique Size by Algorithm')
ax1.tick_params(axis='x', rotation=45)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Plot 2: Execution time (log scale)
ax2.bar(valid_algos, times, color='coral')
ax2.set_xlabel('Algorithm')
ax2.set_ylabel('Time (ms, log scale)')
ax2.set_title('Execution Time by Algorithm')
ax2.set_yscale('log')
ax2.tick_params(axis='x', rotation=45)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Export Results to CSV

In [ ]:
import csv

output_file = 'benchmark_results.csv'

with open(output_file, 'w', newline='') as f:
    fieldnames = ['algorithm', 'vertices', 'edges', 'density', 'clique_size', 'time_ms', 'valid']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    
    writer.writeheader()
    for r in results:
        writer.writerow({
            'algorithm': r['algorithm'],
            'vertices': r['vertices'],
            'edges': r['edges'],
            'density': f"{r['density']:.6f}",
            'clique_size': r['clique_size'],
            'time_ms': f"{r['time_ms']:.2f}",
            'valid': r['valid']
        })

print(f"✓ Results exported to {output_file}")

# Download the file
from google.colab import files
files.download(output_file)

## Performance Comparison: Colab vs Your Laptop

### Your Laptop (Linux Mint):
- **CPU**: Intel i7-13650HX (14 cores, up to 4.9 GHz)
- **RAM**: 16 GB
- **Implementation**: C++ (compiled, native)
- **Expected**: ~3-5x faster than Colab Free

### Google Colab Free:
- **CPU**: ~2 Intel Xeon cores (2.2 GHz)
- **RAM**: ~12 GB
- **Implementation**: Python (interpreted)
- **Expected**: Slower, but still usable for medium graphs

### Speed Comparison:
| Task | Your Laptop (C++) | Colab (Python) |
|------|-------------------|----------------|
| Greedy (1000 vertices) | ~5 ms | ~50 ms |
| Tomita (100 vertices) | ~10 ms | ~100 ms |
| Large heuristics | Seconds | 10-30 seconds |

**Recommendation**: 
- Use your laptop for production runs
- Use Colab for quick experiments, sharing, or when away from your machine

## Tips for Using on Colab

1. **Runtime**: Keep sessions alive (Premium: longer runtime)
2. **GPU**: Not needed for this (CPU-only algorithms)
3. **Large graphs**: May timeout on Colab Free (use smaller datasets)
4. **Memory**: Colab has ~12GB RAM (your laptop: 16GB)
5. **Save results**: Download CSV files before session ends

## Next Steps

1. Upload your SAT-generated graph (`random_3sat_large.txt`)
2. Run the benchmark
3. Compare results with your laptop benchmarks
4. Download the CSV for analysis